In [1]:
import domolibrary.client.get_data as gd
import domolibrary.client.DomoError as dmde

In [8]:
import domolibrary.client.DomoAuth as dmda
import os

#auth into domo instance
auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_ACCESS_TOKEN"],
    domo_instance=os.environ["DOMO_INSTANCE"],
)

await auth.who_am_i()

# workflow variables
model_id = "48707704-213c-4c82-8a7d-69505b50a8de"
version_id = "1.0.7"
# starting_block = "Start%20HelpDeskMafia"

### Get domo workflow information

In [9]:
async def get_workflow(
    auth: dmda.DomoAuth, model_id, version_id, debug_api: bool = False
):

    url = f"https://{auth.domo_instance}.domo.com/api/workflow/v1/models/{model_id}/versions/{version_id}"
    res = await gd.get_data(auth=auth, method="GET", url=url, debug_api=debug_api)

    if not res.is_success:
        raise dmde.RouteError(res=res)

    return res

In [10]:
res = await get_workflow(
    auth=auth, model_id=model_id, version_id=version_id, debug_api=False
)

from pprint import pprint

pprint(res.response["savedJson"])

('{"version":2,"designElements":[{"id":"rootNode","type":"rootNode","position":{"x":60,"y":30},"data":{"dimensions":{"width":200,"height":60},"title":"Start '
 'HelpDeskMafia","description":"","type":"Start","_designNode":"rootNode","isFormStart":true,"input":[{"acceptsInput":true,"children":[],"customMappingType":"form_start","dataType":"text","datasetMapping":null,"displayName":"channel_id","entitySubType":null,"fieldOptionsMappedTo":null,"fieldOptionsValue":null,"flag":"input","formFieldId":"f8fd5581-9f00-493e-8436-050f2a7dc5d5","formFieldType":null,"id":"mLIkhJAYwdDGwfB","isList":false,"mappedTo":"AYTjlyjbwFiAdPY","paramName":"channel_id","required":true,"value":null,"visible":true,"configType":"forms","useExternalValues":false},{"acceptsInput":false,"children":[],"customMappingType":"form_start","dataType":"text","datasetMapping":null,"displayName":"message_id","entitySubType":null,"fieldOptionsMappedTo":null,"fieldOptionsValue":null,"flag":"input","formFieldId":"5aab7003-af7d-4d1

### Trigger domo workflows

In [11]:
import domolibrary.client.get_data as gd
import domolibrary.client.DomoError as dmde


def generate_trigger_workflow_body(starting_tile, model_id, version_id, execution_params : dict):
    return {
        "messageName": starting_tile,
        "version": version_id,
        "modelId": model_id,
        "data": execution_params,
    }


async def trigger_workflow(
    auth: dmda.DomoAuth,
    starting_tile: str,
    model_id: str,
    version_id: str,
    execution_parameters: dict = None,
    debug_api: bool = False,
):
    body = generate_trigger_workflow_body(
        starting_tile=starting_tile,
        model_id=model_id,
        execution_params=execution_parameters,
        version_id=version_id
    )

    url = f"https://{auth.domo_instance}.domo.com/api/workflow/v1/instances/message"

    res = await gd.get_data(
        method="POST", url=url, body=body, auth=auth, debug_api=debug_api
    )

    if not res.is_success:
        raise dmde.RouteError(res=res)

    return res

### Sample trigger workflows
* Requires execution parameters (question that will be asked of agent)
* Requires starting tile name

In [12]:
execution_parameters = {"question": "what is a beast mode"}
starting_tile = "Start HelpDeskMafia"

res = await trigger_workflow(
    auth=auth,
    starting_tile=starting_tile,
    model_id=model_id,
    version_id=version_id,
    execution_parameters= execution_parameters,
    debug_api=False,
)
res.response


RouteError: 🛑  RouteError 🛑 - function: get_traceback || status 400 || Bad Request at domo-community

### Get domo workflow trigger history and executions

In [13]:
async def get_workflow_trigger_history(auth : dmda.DomoAuth, model_id, debug_api : bool = False):

    url = f'https://{auth.domo_instance}.domo.com/api/workflow/v2/executions/{model_id}'

    res = await gd.get_data(auth=auth, method="GET", url=url, debug_api=debug_api)

    if not res.is_success:
        raise dmde.RouteError(res=res)
    
    return res


In [14]:
try:
    await get_workflow_trigger_history(auth = auth, model_id= model_id, debug_api= True)

except Exception as e:
    pass

🐛 debugging get_data
{'body': None,
 'function_name': 'get_traceback',
 'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json, text/plain',
             'x-domo-developer-token': '83ece44f1451d4b581e1191f98cd411164f0b5b6ad2755b3'},
 'method': 'GET',
 'params': None,
 'parent_class': None,
 'url': 'https://domo-community.domo.com/api/workflow/v2/executions/48707704-213c-4c82-8a7d-69505b50a8de'}
get_data: no body
get_data_response <Response [404 Not Found]>


In [ ]:
async def get_workflow_executions(auth : dmda.DomoAuth,
                                  model_id: str,
                                  version_id: str,
                                  debug_api : bool = False
                                  ):
    params = {"modelId" : model_id,
            #   "triggerTypes" : "ALERT,API,APP_STUDIO,CUSTOM_APP,MANUAL,TIMER,WORKFLOW"
              "version" : version_id,
              # "status"  : "IN_PROGRESS"
    }
    
    url = f"https://{auth.domo_instance}.domo.com/api/workflow/v2/executions"

    res = await gd.get_data(auth=auth, method="GET", url=url, debug_api=debug_api, params = params)

    if not res.is_success:
        raise dmde.RouteError(res=res)
    
    return res

    

res = await get_workflow_executions(auth = auth,
                                  model_id = model_id,
                                  version_id = version_id,
                                  debug_api = True
)

res.response

# Domo FileSets Routes

### get one domo FileSet

In [43]:
async def get_fileset(
        auth: dmda.DomoAuth, domo_fileset_id, debug_api: bool = False
):
    url = f'https://{auth.domo_instance}.domo.com/api/files/v1/filesets/{domo_fileset_id}'
    res = await gd.get_data(auth=auth, method='GET', url=url, debug_api=debug_api)

    if not res.is_success:
        raise dmde.RouteError(res=res, debug_api=debug_api)
    
    return res


In [44]:
# Sample implementation of get fileset
DOMO_FILESET_ID = '30d51a7e-15f9-4dce-8284-be392c392f2d'
await get_fileset(auth=auth, domo_fileset_id = DOMO_FILESET_ID, debug_api=False)

ResponseGetData(status=200, response={'id': '30d51a7e-15f9-4dce-8284-be392c392f2d', 'name': 'Test AF', 'description': 'Upload photos', 'aiEnabled': False, 'batchType': 'INCREMENTAL', 'connector': 'DOMO', 'created': '2025-03-17T17:00:12.836646Z', 'createdBy': 2072616249, 'updated': '2025-03-17T17:00:15.828361Z', 'updatedBy': 2072616249, 'owner': '2072616249', 'accountId': 0, 'permission': 'OWNER', 'size': 922503, 'imageCount': 2, 'audioCount': 0, 'videoCount': 0, 'documentCount': 0, 'textCount': 0, 'otherCount': 0}, is_success=True, parent_class=None)

### get fileset files

In [ ]:
async def get_fileset_files(
        auth : dmda.DomoAuth, domo_fileset_id, body: dict = {}, debug_api: bool = False
):
    url = f'https://{auth.domo_instance}.domo.com/api/files/v1/filesets/{domo_fileset_id}/files/search?directoryPath=&immediateChildren=true'
    
    if not body:
        # default body will pull all files within the given fileset_id
        body = {"fieldSort":[{"field":"created","order":"DESC"}],"filters":[],"dateFilters":[]}

    res = await gd.get_data(auth=auth, method='POST', url=url, body=body, debug_api=debug_api)
    if not res.is_success:
        raise dmde.RouteError(res=res)
    
    return res


In [42]:
# Sample implementation of get fileset files
DOMO_FILESET_ID = '30d51a7e-15f9-4dce-8284-be392c392f2d'
res = await get_fileset_files(auth=auth, domo_fileset_id = DOMO_FILESET_ID, debug_api=False)
res.response['files']

[{'id': 'dc9cf268-e88e-4031-b867-5f8e44598fbd',
  'path': '2.jpg',
  'name': '2.jpg',
  'fileType': 'IMAGE',
  'contentType': 'image/jpeg',
  'size': 745364,
  'created': '2025-03-17T17:00:15.606074Z',
  'createdBy': 2072616249},
 {'id': '27800c7f-61b2-4e57-b600-fd5032de73cf',
  'path': 'IMG_4220.jpg',
  'name': 'IMG_4220.jpg',
  'fileType': 'IMAGE',
  'contentType': 'image/jpeg',
  'size': 177139,
  'created': '2025-03-17T17:00:14.059412Z',
  'createdBy': 2072616249}]